In [1]:
# DSC540,Summer 2023 - T302 Data Preparation(2237-1)
# Assignment: Project Milestone 02
# Author by:  Debabrata Mishra
# Date: 2023-07-02

# Topic - Credit Card Transactional & Demographic Data

# Milestone 2 Tasks

Perform at least 5 data transformation and/or cleansing steps to your flat file data. The below examples are not required - they are just potential transformations you could do. If your data doesn't work for these scenarios, complete different transformations. You can do the same transformation multiple times if needed to clean your data. The goal is a clean dataset at the end of the milestone.

    Replace Headers
    Format data into a more readable format
    Identify outliers and bad data
    Find duplicates
    Fix casing or inconsistent values
    Conduct Fuzzy Matching

Make sure you clearly label each transformation (Step #1, Step #2, etc.) in your code and describe what it is doing in 1-2 sentences. You can submit a Jupyter Notebook or a PDF of your code. If you submit a .py file you need to also include a PDF or attachment of your results.

# Cleaning and Formatting of FraudTrain.csv (Flat File Source)

## Flat File

Descrition: 

Simulated credit card transaction dataset containing legitimate and fraud transactions from the duration 1st Jan 2019 - 31st Dec 2020. It covers credit cards of 1000+ customers doing transactions across 693 unique merchant ids. It has 23 columns and transactional information along with demographic details of Merchants and Card Holders.

Link: https://www.kaggle.com/code/nathanxiang/credit-card-fraud-analysis-and-modeling/input?select=fraudTrain.csv

I have downloaded that file to local folder and then executed below steps for cleaning & formatting actvities.

Ethical implications :  should always be considered when working with sensitive data like credit card fraud data. Proper handling and protection of the data are crucial. Access to the data should be restricted to authorized personnel only, and measures should be in place to prevent unauthorized access or theft. The transformed data should be used solely for legitimate purposes such as fraud detection, prevention, or investigation, and should not be used for any illegal or unethical activities. It is also essential to securely delete the data once it is no longer needed to prevent any further exposure of sensitive information.

I will implement boolean indexing methid to identify the values in the 'amt' column that fall outside the specified range. These outliers should be carefully inspected to ensure their accuracy and determine whether they should be considered valid data points or corrected. It would be done when I will connect this with web/HTML data for range reference.Additionally, metrchant address will be derived when I will connect this data with API

As part of Project Milestone 2: I have considered below transformations. 

- Createing and rename header fields

- Createing new variables which required for future calculations like customer age , day, date, month, hour, weekdayfrom transaction date/time field

- Creat Masked the account number and BIN from account number and then drop account number colmun

- Format amount values to 2 decimal points

- Drop columns that I won't be use for any of my planned analysis like dob ( custome age will be used), city_pop, customer latitude and lognitude 

- Identify outliers using IQR

- Identify  outliers using Z-score and drop those records (if any)

- check for duplicate and drop those duplicates (if any)

- Data accuracy check for amount field

- Check for BIN occurance as those help in understanding BOT attacks


In [2]:
#Load the Necessary Libraries
import requests as r
import pandas as pd
import xlrd
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt

In [3]:
txn_data = pd.read_csv('fraudTrain.csv', sep=",")
txn_data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [4]:
# Overview of the structure and characteristics
txn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [5]:
# 01- Rename the headers with meaningful names.
txn_data.rename(columns = {'Unnamed: 0':'row_id', 'cc_num':'accountNumber', 'amt':'amount','merchant':'merch_name'}, inplace = True)
txn_data.head()

,row_id,trans_date_trans_time,accountNumber,merch_name,category,amount,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [6]:
# 02- Create some new colmuns linked with timestamp and customer date of birth which could be used for calculation(s)
#     format the timestamp fields

# Create variables from trasnaction date/time for future calculations
txn_data['trans_date_trans_time'] = pd.to_datetime(txn_data['trans_date_trans_time'],errors='coerce')

txn_data['year']=txn_data['trans_date_trans_time'].dt.year
txn_data['month']=txn_data['trans_date_trans_time'].dt.strftime('%b')
txn_data['month']=txn_data['trans_date_trans_time'].dt.month
txn_data['day']=txn_data['trans_date_trans_time'].dt.day
txn_data['hour']=txn_data['trans_date_trans_time'].dt.hour
txn_data['weekday']=txn_data['trans_date_trans_time'].dt.strftime('%a')
txn_data['dayofYear']=txn_data['trans_date_trans_time'].dt.dayofyear

# Create a variable age of customer on day of transaction

txn_data['txn_date']=txn_data['trans_date_trans_time'].dt.strftime('%Y-%m-%d')
txn_data['txn_date']=pd.to_datetime(txn_data['txn_date'])
txn_data['dob']=pd.to_datetime(txn_data['dob'])
txn_data["customer_age"] =txn_data["txn_date"]-txn_data["dob"]
txn_data["customer_age"] =txn_data["customer_age"].astype('timedelta64[Y]')

# Transform the unix time into timestamp format
txn_data['unix_time'] = pd.to_datetime(txn_data['unix_time'], unit='s')

txn_data.head()

,row_id,trans_date_trans_time,accountNumber,merch_name,category,amount,first,last,gender,street,...,merch_long,is_fraud,year,month,day,hour,weekday,dayofYear,txn_date,customer_age
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,-82.048315,0,2019,1,1,0,Tue,1,2019-01-01,30.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,-118.186462,0,2019,1,1,0,Tue,1,2019-01-01,40.0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,-112.154481,0,2019,1,1,0,Tue,1,2019-01-01,56.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,-112.561071,0,2019,1,1,0,Tue,1,2019-01-01,51.0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,-78.632459,0,2019,1,1,0,Tue,1,2019-01-01,32.0


In [7]:
# 03 - Mask the account number , Create the BIN (Bank Identification Number) and then drop the clear account number

# Define a function to mask the account number
def mask_account_number(accountNumber):
    return accountNumber[:4] + '*' * (len(accountNumber) - 8) + accountNumber[-4:]

# Apply the masking function to the 'accountNumber' column

# Convert AccountNumber column to string
txn_data['accountNumber'] = txn_data['accountNumber'].astype(str)
txn_data['masked_accountNumber'] = txn_data['accountNumber'].apply(mask_account_number)

# Extract the BIN from the masked account number
txn_data['BIN'] = txn_data['accountNumber'].str[:6]

# Drop the 'accountNumber' column
txn_data = txn_data.drop('accountNumber', axis=1)

txn_data.head()

,row_id,trans_date_trans_time,merch_name,category,amount,first,last,gender,street,city,...,year,month,day,hour,weekday,dayofYear,txn_date,customer_age,masked_accountNumber,BIN
0,0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,2019,1,1,0,Tue,1,2019-01-01,30.0,2703********2095,270318
1,1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,2019,1,1,0,Tue,1,2019-01-01,40.0,6304****7322,630423
2,2,2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,2019,1,1,0,Tue,1,2019-01-01,56.0,3885******7661,388594
3,3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,2019,1,1,0,Tue,1,2019-01-01,51.0,3534********0240,353409
4,4,2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,2019,1,1,0,Tue,1,2019-01-01,32.0,3755*******3984,375534


In [8]:
# 04 - Format the amount to float with 2 decimal points
txn_data['amount'] = txn_data['amount'].round(2)
txn_data.head()

,row_id,trans_date_trans_time,merch_name,category,amount,first,last,gender,street,city,...,year,month,day,hour,weekday,dayofYear,txn_date,customer_age,masked_accountNumber,BIN
0,0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,2019,1,1,0,Tue,1,2019-01-01,30.0,2703********2095,270318
1,1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,2019,1,1,0,Tue,1,2019-01-01,40.0,6304****7322,630423
2,2,2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,2019,1,1,0,Tue,1,2019-01-01,56.0,3885******7661,388594
3,3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,2019,1,1,0,Tue,1,2019-01-01,51.0,3534********0240,353409
4,4,2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,2019,1,1,0,Tue,1,2019-01-01,32.0,3755*******3984,375534


In [9]:
# 05- Check for the missing values for all columns

for col in txn_data.columns:
    count = txn_data[col].isnull().sum()
    if count>0:
        print("{} has {} missing value(s).".format(col,miss))
    else:
        print("{} has no missing values.".format(col))

row_id has no missing values.
trans_date_trans_time has no missing values.
merch_name has no missing values.
category has no missing values.
amount has no missing values.
first has no missing values.
last has no missing values.
gender has no missing values.
street has no missing values.
city has no missing values.
state has no missing values.
zip has no missing values.
lat has no missing values.
long has no missing values.
city_pop has no missing values.
job has no missing values.
dob has no missing values.
trans_num has no missing values.
unix_time has no missing values.
merch_lat has no missing values.
merch_long has no missing values.
is_fraud has no missing values.
year has no missing values.
month has no missing values.
day has no missing values.
hour has no missing values.
weekday has no missing values.
dayofYear has no missing values.
txn_date has no missing values.
customer_age has no missing values.
masked_accountNumber has no missing values.
BIN has no missing values.


In [10]:
# 06 - Drop/ Delete the 'city_pop' and 'dob' column.
columns_to_drop = ['dob', 'city_pop', 'lat','long']
txn_data = txn_data.drop(columns_to_drop, axis=1)
txn_data.head()

,row_id,trans_date_trans_time,merch_name,category,amount,first,last,gender,street,city,...,year,month,day,hour,weekday,dayofYear,txn_date,customer_age,masked_accountNumber,BIN
0,0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,2019,1,1,0,Tue,1,2019-01-01,30.0,2703********2095,270318
1,1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,2019,1,1,0,Tue,1,2019-01-01,40.0,6304****7322,630423
2,2,2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,2019,1,1,0,Tue,1,2019-01-01,56.0,3885******7661,388594
3,3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,2019,1,1,0,Tue,1,2019-01-01,51.0,3534********0240,353409
4,4,2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,2019,1,1,0,Tue,1,2019-01-01,32.0,3755*******3984,375534


In [11]:
# 07- Identify outliers and bad data
Q1 = txn_data['amount'].quantile(0.25)
Q3 = txn_data['amount'].quantile(0.75)
IQR = Q3 - Q1
outliers = txn_data[(txn_data['amount'] < (Q1 - 1.5 * IQR)) |(txn_data['amount'] > (Q3 + 1.5 * IQR))]
print("Outliers:\n", outliers)

Outliers:
           row_id trans_date_trans_time  \
2              2   2019-01-01 00:00:51   
9              9   2019-01-01 00:06:01   
16            16   2019-01-01 00:10:49   
17            17   2019-01-01 00:10:58   
36            36   2019-01-01 00:26:22   
...          ...                   ...   
1296547  1296547   2020-06-21 11:07:51   
1296557  1296557   2020-06-21 11:14:04   
1296590  1296590   2020-06-21 11:32:55   
1296603  1296603   2020-06-21 11:36:41   
1296657  1296657   2020-06-21 12:07:20   

                                      merch_name       category  amount  \
2                           fraud_Lind-Buckridge  entertainment  220.11   
9              fraud_Schoen, Kuphal and Nitzsche    grocery_pos  198.39   
16                        fraud_Lebsack and Sons       misc_net  327.00   
17                            fraud_Mayert Group   shopping_pos  341.67   
36                         fraud_Heidenreich PLC    grocery_pos  207.36   
...                               

In [12]:
# 08- Identify outliers and remove those records

size_prev = txn_data.shape

# Calculate z-scores for 'amount' column
z_scores = np.abs((txn_data['amount'] - txn_data['amount'].mean()) / txn_data['amount'].std())

# Set a threshold for outlier detection (e.g., z-score > 3)
lower_threshold = -3
upper_threshold = 3

# Remove rows with outliers in 'amount' column
txn_data = txn_data[(z_scores >= lower_threshold) & (z_scores <= upper_threshold)]

# Display the updated dataset
size_after = txn_data.shape
print(f"The size of previous data was - {size_prev} rows and the size of the new one is - {size_after} rows")


The size of previous data was - (1296675, 28) rows and the size of the new one is - (1283937, 28) rows


In [13]:
# 09- Identify any duplicate rows
duplicates = txn_data[txn_data.duplicated()]
print("Duplicates:\n", duplicates)

Duplicates:
 Empty DataFrame
Columns: [row_id, trans_date_trans_time, merch_name, category, amount, first, last, gender, street, city, state, zip, job, trans_num, unix_time, merch_lat, merch_long, is_fraud, year, month, day, hour, weekday, dayofYear, txn_date, customer_age, masked_accountNumber, BIN]
Index: []

[0 rows x 28 columns]


In [14]:
# 10 - Data accuracy and Check for negative 'amount' values

negative_amounts = txn_data[txn_data['amount'] < 0]
if not negative_amounts.empty:
    # Replace negative values with NaN
    txn_data.loc[txn_data['amount'] < 0, 'amount'] = np.nan
    print("Negative amounts found and replaced with NaN.")
else:
    print("No negative amounts found.")

No negative amounts found.


In [15]:
# 11 - Get the unique values of the 'BIN' field
unique_bins = txn_data['BIN'].unique()

# Display the unique BIN values
print(unique_bins)

['270318' '630423' '388594' '353409' '375534' '476726' '300746' '601136'
 '492271' '272083' '464289' '377234' '180042' '555985' '351486' '601199'
 '601186' '356542' '234824' '495682' '446977' '230533' '180048' '630441'
 '442878' '343464' '374930' '433423' '422599' '426005' '180094' '459973'
 '630412' '271220' '374125' '349612' '359821' '213141' '305612' '478991'
 '676372' '300114' '464225' '180067' '213126' '226673' '355362' '351105'
 '411026' '426577' '358379' '213193' '604955' '472583' '304042' '213178'
 '235627' '356873' '450914' '354756' '443309' '272043' '471656' '353691'
 '352566' '470786' '466631' '224873' '451282' '350150' '302730' '355888'
 '304876' '571465' '378006' '341283' '448113' '468726' '545677' '498113'
 '601189' '653462' '376028' '571365' '351875' '357757' '213112' '359232'
 '468352' '350208' '651134' '419847' '442348' '351823' '659325' '601149'
 '414923' '639046' '223561' '510480' '180040' '412976' '401810' '447432'
 '303762' '348789' '180014' '226080' '497942' '3757

In [16]:
# 12- Get the count of occurrences for each unique value in the 'BIN' field
bin_counts = txn_data['BIN'].value_counts()

# Display the count of occurrences
print(bin_counts)

601136    6133
601110    5601
213161    4538
601138    4023
180067    3553
          ... 
271601       2
353521       1
349813       1
657777       1
601173       1
Name: BIN, Length: 960, dtype: int64


In [17]:
# After all Transformation and the final transactional data
txn_data

,row_id,trans_date_trans_time,merch_name,category,amount,first,last,gender,street,city,...,year,month,day,hour,weekday,dayofYear,txn_date,customer_age,masked_accountNumber,BIN
0,0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,2019,1,1,0,Tue,1,2019-01-01,30.0,2703********2095,270318
1,1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,2019,1,1,0,Tue,1,2019-01-01,40.0,6304****7322,630423
2,2,2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,2019,1,1,0,Tue,1,2019-01-01,56.0,3885******7661,388594
3,3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,2019,1,1,0,Tue,1,2019-01-01,51.0,3534********0240,353409
4,4,2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,2019,1,1,0,Tue,1,2019-01-01,32.0,3755*******3984,375534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,1296670,2020-06-21 12:12:08,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,Hatch,...,2020,6,21,12,Sun,173,2020-06-21,58.0,3026******4123,302635
1296671,1296671,2020-06-21 12:12:19,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,Tuscarora,...,2020,6,21,12,Sun,173,2020-06-21,40.0,6011********6997,601114
1296672,1296672,2020-06-21 12:12:32,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,High Rolls Mountain Park,...,2020,6,21,12,Sun,173,2020-06-21,52.0,3514********4695,351486
1296673,1296673,2020-06-21 12:13:36,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,Manderson,...,2020,6,21,12,Sun,173,2020-06-21,39.0,2720********6919,272001


In [18]:
txn_data.to_csv('final_txn_data.csv', index=False)